In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [8]:
ROOT = '/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/'

In [12]:
c9_inp_df = pd.read_csv(f"{ROOT}Excitatory_C9ALSvsCTRL/APAlog_res_metadata_added.tsv", sep='\t')
sA_inp_df = pd.read_csv(f"{ROOT}Excitatory_sALSvsCTRL/APAlog_res_metadata_added.tsv", sep='\t')
sA_inp_df

,transcript,p_devtest,fdr_p_devtest,ref_site,alt_site,b_intercept,p_intercept,b_ConditionALS_pathology,p_ConditionALS_pathology,NA.,NA..1,strand,multiplyer,LFC_PA_Usage,negative_logFDR,switch_width,bed,correction_multiplyer,switch_name,sig_LFC_PA_Usage
0,AACS,9.530774e-01,1.000000,chr12:125140928:+,chr12:125143316:+,1.380056,1.508869e-17,0.012500,0.953068,NaN,NaN,+,1,0.012500,0.020876,2388,"chr12,125140928,125143316",1,chr12:AACS:125140928:125143316:+,0.000000
1,AAGAB,2.506463e-07,0.000002,chr15:67201672:-,chr15:67202710:-,0.531540,1.436636e-02,-1.492002,0.000001,NaN,NaN,-,-1,1.492002,5.933483,1038,"chr15,67201672,67202710",1,chr15:AAGAB:67201672:67202710:-,1.492002
2,AAGAB,2.506463e-07,0.000002,chr15:67201672:-,chr15:67217079:-,0.160773,4.929285e-01,-1.121235,0.000448,NaN,NaN,-,-1,1.121235,3.349091,15407,"chr15,67201672,67217079",1,chr15:AAGAB:67201672:67217079:-,1.121235
3,AAGAB,2.506463e-07,0.000002,chr15:67202710:-,chr15:67217079:-,-0.370767,7.289612e-02,0.370767,0.265314,NaN,NaN,-,-1,-0.370767,0.576240,14369,"chr15,67202710,67217079",1,chr15:AAGAB:67202710:67217079:-,0.000000
4,AAK1,1.000000e+00,1.000000,chr2:69457997:-,chr2:69461526:-,0.140089,1.358893e-01,0.137837,0.251954,NaN,NaN,-,-1,-0.137837,0.598678,3529,"chr2,69457997,69461526",1,chr2:AAK1:69457997:69461526:-,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,ZZEF1,1.000000e+00,1.000000,chr17:4006020:-,chr17:4116972:-,1.375823,9.943161e-09,-19.977882,0.990093,NaN,NaN,-,-1,19.977882,0.004324,110952,"chr17,4006020,4116972",1,chr17:ZZEF1:4006020:4116972:-,0.000000
27533,ZZEF1,1.000000e+00,1.000000,chr17:4008624:-,chr17:4021776:-,0.055333,8.292346e-01,18.039428,0.991285,NaN,NaN,-,-1,-18.039428,0.003801,13152,"chr17,4008624,4021776",1,chr17:ZZEF1:4008624:4021776:-,0.000000
27534,ZZEF1,1.000000e+00,1.000000,chr17:4008624:-,chr17:4116972:-,1.068251,5.404068e-07,NaN,NaN,NaN,NaN,-,-1,NaN,NaN,108348,"chr17,4008624,4116972",1,chr17:ZZEF1:4008624:4116972:-,NaN
27535,ZZEF1,1.000000e+00,1.000000,chr17:4021776:-,chr17:4116972:-,1.012918,1.234933e-06,-18.107679,0.985578,NaN,NaN,-,-1,18.107679,0.006309,95196,"chr17,4021776,4116972",1,chr17:ZZEF1:4021776:4116972:-,0.000000


In [13]:
# write a bed file for FIRE analysis, we only need the switch_name and LFC_PA_Usage columns
# mkdir ROOT/FIRE_analysis_data if not exist
if not os.path.exists(ROOT + 'FIRE_analysis_data'):
    os.mkdir(ROOT + 'FIRE_analysis_data')
c9_inp_df[['switch_name', 'LFC_PA_Usage']].to_csv(ROOT + 'FIRE_analysis_data/' + 'Excitatory_C9ALS_APAs.bed', sep='\t', header=False, index=False)
sA_inp_df[['switch_name', 'LFC_PA_Usage']].to_csv(ROOT + 'FIRE_analysis_data/' + 'Excitatory_sALS_APAs.bed', sep='\t', header=False, index=False)

In [25]:
# lets make bed file with switch_name, chr12,125140928,125143316 and strand column
# bed:  chr, start + 1000 , start - 1000, strand, switch_name
# bed:  chr, end + 1000 , end - 1000, strand, switch_name

bed = pd.DataFrame()
for i, row in c9_inp_df.iterrows():
    switch_name = row['switch_name']
    # split the switch name and get chr, start, end
    chr, gene_name, start, end, strand = switch_name.split(':')
    start = int(start)
    end = int(end)
    # make bed file
    bed = bed.append({'chr': chr, 'start': int(start - 250), 'end': int(start + 250), 'strand': strand, 'switch_name': switch_name}, ignore_index=True)
    bed = bed.append({'chr': chr, 'start': int(end - 250), 'end': int(end + 250), 'strand': strand, 'switch_name': switch_name}, ignore_index=True)
bed['start'] = bed['start'].astype(int)
bed['end'] = bed['end'].astype(int)
bed.to_csv(f'{ROOT}FIRE_analysis_data/c9_apa_hnrnpc.bed', sep='\t', index=False, header=False)


In [26]:
bed = pd.DataFrame()
for i, row in sA_inp_df.iterrows():
    switch_name = row['switch_name']
    # split the switch name and get chr, start, end
    chr, gene_name, start, end, strand = switch_name.split(':')
    start = int(start)
    end = int(end)
    # make bed file
    bed = bed.append({'chr': chr, 'start': int(start - 250), 'end': int(start + 250), 'strand': strand, 'switch_name': switch_name}, ignore_index=True)
    bed = bed.append({'chr': chr, 'start': int(end - 250), 'end': int(end + 250), 'strand': strand, 'switch_name': switch_name}, ignore_index=True)
bed['start'] = bed['start'].astype(int)
bed['end'] = bed['end'].astype(int)
bed.to_csv(f'{ROOT}FIRE_analysis_data/sALS_apa_hnrnpc.bed', sep='\t', index=False, header=False)

## FIRE commands example:
`export TEISERDIR=/avicenna/bins/TEISERv1.1/`

`perl /avicenna/bins/TEISERv1.1/run_mi_gene_list.pl --expfile=Excitatory_C9ALS_APAs.bed --genefile=c9ALS_hnrnpc_loci.bed --exptype=continuous --ebins=10 --doremovedups=0 --doremoveextra=0`


## first keep the PAs that there is hnrnpc motif around it +- 1000 bases
`bedtools intersect -a c9_apa_hnrnpc.bed -b /avicenna/hani/clearCLIP/HNRNPC/HNRNPC_clearCLIP.pool.tag.uniq.del.CIMS.fdr10.bgfilter.bed > c9ALS_hnrnpc_loci.bed`
`cat c9ALS_hnrnpc_loci.bed | awk '{print $5}' > c9ALS_hnrnpc_loci_hits.bed`
`bedtools intersect -a sALS_apa_hnrnpc.bed -b /avicenna/hani/clearCLIP/HNRNPC/HNRNPC_clearCLIP.pool.tag.uniq.del.CIMS.fdr10.bgfilter.bed > sALS_hnrnpc_loci.bed`

`cat sALS_hnrnpc_loci.bed | awk '{print $5}' > sALS_hnrnpc_loci_hits.bed`


## Lets try  only proximal sites

In [28]:
c9_inp_df = pd.read_csv(f"{ROOT}Excitatory_C9ALSvsCTRL/APAlog_res_metadata_added.tsv", sep='\t')
sA_inp_df = pd.read_csv(f"{ROOT}Excitatory_sALSvsCTRL/APAlog_res_metadata_added.tsv", sep='\t')
sA_inp_df

,transcript,p_devtest,fdr_p_devtest,ref_site,alt_site,b_intercept,p_intercept,b_ConditionALS_pathology,p_ConditionALS_pathology,NA.,NA..1,strand,multiplyer,LFC_PA_Usage,negative_logFDR,switch_width,bed,correction_multiplyer,switch_name,sig_LFC_PA_Usage
0,AACS,9.530774e-01,1.000000,chr12:125140928:+,chr12:125143316:+,1.380056,1.508869e-17,0.012500,0.953068,NaN,NaN,+,1,0.012500,0.020876,2388,"chr12,125140928,125143316",1,chr12:AACS:125140928:125143316:+,0.000000
1,AAGAB,2.506463e-07,0.000002,chr15:67201672:-,chr15:67202710:-,0.531540,1.436636e-02,-1.492002,0.000001,NaN,NaN,-,-1,1.492002,5.933483,1038,"chr15,67201672,67202710",1,chr15:AAGAB:67201672:67202710:-,1.492002
2,AAGAB,2.506463e-07,0.000002,chr15:67201672:-,chr15:67217079:-,0.160773,4.929285e-01,-1.121235,0.000448,NaN,NaN,-,-1,1.121235,3.349091,15407,"chr15,67201672,67217079",1,chr15:AAGAB:67201672:67217079:-,1.121235
3,AAGAB,2.506463e-07,0.000002,chr15:67202710:-,chr15:67217079:-,-0.370767,7.289612e-02,0.370767,0.265314,NaN,NaN,-,-1,-0.370767,0.576240,14369,"chr15,67202710,67217079",1,chr15:AAGAB:67202710:67217079:-,0.000000
4,AAK1,1.000000e+00,1.000000,chr2:69457997:-,chr2:69461526:-,0.140089,1.358893e-01,0.137837,0.251954,NaN,NaN,-,-1,-0.137837,0.598678,3529,"chr2,69457997,69461526",1,chr2:AAK1:69457997:69461526:-,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,ZZEF1,1.000000e+00,1.000000,chr17:4006020:-,chr17:4116972:-,1.375823,9.943161e-09,-19.977882,0.990093,NaN,NaN,-,-1,19.977882,0.004324,110952,"chr17,4006020,4116972",1,chr17:ZZEF1:4006020:4116972:-,0.000000
27533,ZZEF1,1.000000e+00,1.000000,chr17:4008624:-,chr17:4021776:-,0.055333,8.292346e-01,18.039428,0.991285,NaN,NaN,-,-1,-18.039428,0.003801,13152,"chr17,4008624,4021776",1,chr17:ZZEF1:4008624:4021776:-,0.000000
27534,ZZEF1,1.000000e+00,1.000000,chr17:4008624:-,chr17:4116972:-,1.068251,5.404068e-07,NaN,NaN,NaN,NaN,-,-1,NaN,NaN,108348,"chr17,4008624,4116972",1,chr17:ZZEF1:4008624:4116972:-,NaN
27535,ZZEF1,1.000000e+00,1.000000,chr17:4021776:-,chr17:4116972:-,1.012918,1.234933e-06,-18.107679,0.985578,NaN,NaN,-,-1,18.107679,0.006309,95196,"chr17,4021776,4116972",1,chr17:ZZEF1:4021776:4116972:-,0.000000


In [29]:
# lets make bed file with switch_name, chr12,125140928,125143316 and strand column
# bed:  chr, start + 1000 , start - 1000, strand, switch_name
# bed:  chr, end + 1000 , end - 1000, strand, switch_name

bed = pd.DataFrame()
for i, row in c9_inp_df.iterrows():
    switch_name = row['switch_name']
    # split the switch name and get chr, start, end
    chr, gene_name, start, end, strand = switch_name.split(':')
    start = int(start)
    end = int(end)
    # make bed file
    if strand == '+':
        bed = bed.append({'chr': chr, 'start': int(start - 250), 'end': int(start + 250), 'strand': strand, 'switch_name': switch_name}, ignore_index=True)
    else:
        bed = bed.append({'chr': chr, 'start': int(end - 250), 'end': int(end + 250), 'strand': strand, 'switch_name': switch_name}, ignore_index=True)
bed['start'] = bed['start'].astype(int)
bed['end'] = bed['end'].astype(int)
bed.to_csv(f'{ROOT}FIRE_analysis_data/c9_apa_hnrnpc.bed', sep='\t', index=False, header=False)


In [30]:
# lets make bed file with switch_name, chr12,125140928,125143316 and strand column
# bed:  chr, start + 1000 , start - 1000, strand, switch_name
# bed:  chr, end + 1000 , end - 1000, strand, switch_name

bed = pd.DataFrame()
for i, row in sA_inp_df.iterrows():
    switch_name = row['switch_name']
    # split the switch name and get chr, start, end
    chr, gene_name, start, end, strand = switch_name.split(':')
    start = int(start)
    end = int(end)
    # make bed file
    if strand == '+':
        bed = bed.append({'chr': chr, 'start': int(start - 250), 'end': int(start + 250), 'strand': strand, 'switch_name': switch_name}, ignore_index=True)
    else:
        bed = bed.append({'chr': chr, 'start': int(end - 250), 'end': int(end + 250), 'strand': strand, 'switch_name': switch_name}, ignore_index=True)
bed['start'] = bed['start'].astype(int)
bed['end'] = bed['end'].astype(int)
bed.to_csv(f'{ROOT}FIRE_analysis_data/sALS_apa_hnrnpc.bed', sep='\t', index=False, header=False)
